# Dataset Preparation

In [1]:
import pandas as pd
import os
import requests, bs4
from bs4 import BeautifulSoup
from fuzzywuzzy import fuzz 
from fuzzywuzzy import process

C:\Users\Antonio\Anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
current_path = os.getcwd()

In [3]:
data_set = pd.read_csv(current_path + "\\Dataset\\precision_agriculture_patents_from_LENS.csv", low_memory=False)

In [4]:
data_set.shape

(84038, 30)

## Data Cleaning

### Removing missing values and duplicates

In [5]:
data_set = data_set.drop_duplicates(subset ="Publication_Number", keep = "last") 

In [6]:
data_set.isnull().sum()

#                                  0
Jurisdiction                       0
Kind                               0
Publication_Number                 0
Lens_ID                            0
Publication_Date                   0
Publication_Year                   0
Application_Number                 0
Application_Date                   0
Priority_Numbers                   0
Earliest_Priority_Date             0
Title                              0
Applicants                         7
Inventors                        103
Owners_(US)                    72268
URL                                0
Type                               0
Has_Full_Text                      0
Cited_by_Patent_Count              0
Simple_Family_Size                 0
Extended_Family_Size               0
Sequence_Count                     0
CPC_Classifications            18892
IPCR_Classifications             634
US_Classifications             49108
NPL_Citation_Count                 0
NPL_Resolved_Citation_Count        0
N

In [7]:
data_set = data_set.dropna(subset=['IPCR_Classifications']).reset_index(drop=True)

In [8]:
data_set.shape

(75243, 30)

### Removing not necessary columns 

In [9]:
data_set.columns

Index(['#', 'Jurisdiction', 'Kind', 'Publication_Number', 'Lens_ID',
       'Publication_Date', 'Publication_Year', 'Application_Number',
       'Application_Date', 'Priority_Numbers', 'Earliest_Priority_Date',
       'Title', 'Applicants', 'Inventors', 'Owners_(US)', 'URL', 'Type',
       'Has_Full_Text', 'Cited_by_Patent_Count', 'Simple_Family_Size',
       'Extended_Family_Size', 'Sequence_Count', 'CPC_Classifications',
       'IPCR_Classifications', 'US_Classifications', 'NPL_Citation_Count',
       'NPL_Resolved_Citation_Count', 'NPL_Resolved_Lens_ID(s)',
       'NPL_Resolved_External_Id(s)', 'NPL_Citations'],
      dtype='object')

In [10]:
columns = ['#', 'Jurisdiction', 'Kind', 'Lens_ID',
       'Publication_Date', 'Application_Number',
       'Application_Date', 'Priority_Numbers', 'Earliest_Priority_Date',
       'Inventors', 'Owners_(US)', 'URL', 'Type',
       'Has_Full_Text', 'Cited_by_Patent_Count', 'Simple_Family_Size',
       'Extended_Family_Size', 'Sequence_Count', 'CPC_Classifications',
       'US_Classifications', 'NPL_Citation_Count',
       'NPL_Resolved_Citation_Count', 'NPL_Resolved_Lens_ID(s)',
       'NPL_Resolved_External_Id(s)', 'NPL_Citations']

In [11]:
data_set = data_set.drop(columns=columns)

In [12]:
data_set.shape

(75243, 5)

### Correct the names of the top 300 applicants e.g. "Mosanto Technology LLC" -> "Monsanto Technology LLC"

In [25]:
%%time
first_300_applicants = list(data_set['Applicants'].value_counts().sort_values(ascending=False).index[:300])
choices = data_set.Applicants.unique()
for i in first_300_applicants:
    a = process.extract(i, choices, limit=100, scorer=fuzz.token_sort_ratio)
    for j in a:
        if j[1]>75:
            data_set.Applicants.replace(j[0], i, inplace = True)

Wall time: 29min 25s


## Data Preprocessing

### Creating 2 columns containing the main class and the main subclass of each patent

In [16]:
def assigne_class_subclass(Ipcr_class):
    ipcr_values = Ipcr_class.split(';;')
    class_values= []
    subclass_values = []
    for ipcr in ipcr_values:
        class_values.append(ipcr[:3])
        subclass_values.append(ipcr[:4])
        
    return max(class_values,key=class_values.count), max(subclass_values,key=subclass_values.count)

In [17]:
data_set['Main_Class'] = ""
data_set['Main_Subclass'] = ""
for i in range(len(data_set)):
    classe, subclass = assigne_class_subclass(data_set.IPCR_Classifications[i])
    data_set.loc[i,"Main_Class"] = classe
    data_set.loc[i,"Main_Subclass"] = subclass

### Getting data with a crawler 

###### We suggest to test the crawler with the given indexes because the execution on the whole dataset it requires days and jump to the slot:  
[ data_crawler = pd.read_csv(current_path+"\\Dataset\\crawler_result.csv") ]

In [ ]:
initial_index = 0
final_index = 5
df = data_set[initial_index:final_index]

In [ ]:
ID_list = df.Publication_Number
URL_list = []
for id_num in ID_list:
    ending_url = id_num
    ending_url = ending_url.replace(" ","")
    ending_url = ending_url.replace("/","")
    URL_list.append("https://patents.google.com/patent/" + ending_url)

In [ ]:
def getting_data_from_patent(id_num, link):
    citation_id_string = ""
    citation_date_string = ""
    cited_id_string = ""
    cited_date_string = ""

    try:
        response = requests.get(link)
    except:
        print("Error URL with: " + link)
        return -1
    
    if(response.status_code == 404):
        print("Status 404: " + link)
        return 404
    
    try:
        patentSoup = bs4.BeautifulSoup(response.text)
        citations = patentSoup.find_all("tr", itemprop="backwardReferencesOrig")
        cited = patentSoup.find_all("tr", itemprop="forwardReferencesOrig")
    except:
        print("Eccezione citations: " + link)
        return 300
    try:        
        abstract = patentSoup.find(name="abstract").text
    except:
        abstract = ""
        
    try:
        claims_section = patentSoup.find_all("section", itemprop="claims")
    except:
        print("Eccezione claim Section: " + link)
        return 300
    
    try:
        claims_occ = claims_section[0].find("span", itemprop="count").text
    except:
        try:
            x = claims_section[0].find("div", itemprop="content")
            claims_occ = len(x.find_all("claim"))
        except:
            claims_occ = 0
        
    for elem in citations:
        pub_num_citation = elem.find(itemprop="publicationNumber").text
        citation_id_string = citation_id_string + pub_num_citation + ";;"
        
        pub_date_citation = elem.find(itemprop="priorityDate").text
        citation_date_string = citation_date_string + pub_date_citation + ";;"
    
    for elem in cited:
        pub_num_cited = elem.find(itemprop="publicationNumber").text
        cited_id_string = cited_id_string + pub_num_cited + ";;"
        
        pub_date_cited = elem.find(itemprop="priorityDate").text
        cited_date_string = cited_date_string + pub_date_cited + ";;"

    abstract = abstract[1:]
    citation_id_string = citation_id_string[:-2]
    citation_date_string = citation_date_string[:-2]
    cited_id_string = cited_id_string[:-2]
    cited_date_string = cited_date_string[:-2]
    
    return id_num, abstract, claims_occ, len(citations), citation_id_string, citation_date_string, len(cited), cited_id_string, cited_date_string 

In [ ]:
attributes = ['Publication_Number', 'Abstract', 'Claims_count', 'Citation_count', 'Citation_ids','Citation_dates', 'Cited_count', 'Cited_ids' ,'Cited_dates']
data_crawler =  pd.DataFrame(columns = attributes)
patent_num = 1 
total_patents = (len(ID_list))
not_found = 0
excep = 0

for id_num in ID_list:
    row = tuple()
    print ("Patent " + str(patent_num) + " of " + str(total_patents))
    ending_url = id_num
    ending_url = ending_url.replace(" ","")
    ending_url = ending_url.replace("/","")
    url = "https://patents.google.com/patent/" + ending_url

    row = getting_data_from_patent(id_num, url)
    if row == -1:
        break
    if row == 404:
        not_found = not_found + 1
    elif row == 300:
        excep = excep + 1
    else: 
        data_crawler = data_crawler.append(pd.Series(row, index=data_crawler.columns ), ignore_index=True)
        
    patent_num = patent_num + 1

In [18]:
data_crawler = pd.read_csv(current_path+"\\Dataset\\crawler_result.csv")

In [19]:
df = pd.merge(data_set, data_crawler, on='Publication_Number', how='inner')

In [20]:
df = df.reset_index(drop=Trueue)

In [21]:
df.shape

(72224, 15)

In [26]:
output_path = current_path +"\\Dataset\\precision_agriculture_patents.csv"
df.to_csv(output_path, index=False) 